# SST detrending
This notebook visualizes results from `SST_generation.py`.

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot
matplotlib.rc_file('../rc_file')

In [ ]:
from paths import path_samoc, path_results, file_ex_ocn_ctrl
from timeseries import IterateOutputCESM
from ab_derivation_SST import DeriveSST as DS
from bc_analysis_fields import AnalyzeField as AF
from bd_analysis_indices import AnalyzeIndex as AI
from xr_regression import xr_quadtrend

## How to detrend?

### Quadratic pointwise detrending
what detrending starting year to choose?

In [ ]:
da = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc', decode_times=False)
# dt = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_sqdt_yrly_ctrl.nc', decode_times=False)
for lat in [400, 1700, 1800, 1900]:
    plt.figure()
    da.sel({'nlon':800, 'nlat':lat}).plot()
#     dt.sel({'nlon':800, 'nlat':lat}).plot()
    xr_quadtrend(da.sel({'nlon':800, 'nlat':lat})[150:]+1.5).plot()
    xr_quadtrend(da.sel({'nlon':800, 'nlat':lat})[100:]+1).plot()
    xr_quadtrend(da.sel({'nlon':800, 'nlat':lat})[40:]+.5).plot()
    xr_quadtrend(da.sel({'nlon':800, 'nlat':lat})).plot()


In [ ]:
da = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc', decode_times=False)

for lat in [50, 250, 300, 350]:
    plt.figure()
    da.sel({'nlon':0, 'nlat':lat}).plot()
#     dt.sel({'nlon':800, 'nlat':lat}).plot()
    xr_quadtrend(da.sel({'nlon':0, 'nlat':lat})[150:]+1.5).plot()
    xr_quadtrend(da.sel({'nlon':0, 'nlat':lat})[100:]+1).plot()
    xr_quadtrend(da.sel({'nlon':0, 'nlat':lat})[40:]+.5).plot()
    xr_quadtrend(da.sel({'nlon':0, 'nlat':lat})).plot()


### scaled GMSST detrending

In [ ]:
# %%time
# 1:32 min for both
for run in ['ctrl', 'lpd']:
    try:
        fn = f'{path_samoc}/SST/GMSST_yrly_{run}.nc'
        assert os.path.exists(fn)
        print(f'file exists: {fn}')
    except:
        DS().generate_yrly_global_mean_SST(run=run)

In [ ]:
for i, run in enumerate(['ctrl', 'lpd']):
    da = xr.open_dataarray(f'{path_samoc}/SST/GMSST_yrly_{run}.nc')
    plt.plot(da.time/365, da- i/2, lw=.5)
    if run=='ctrl':  x = da[40:]
    else:            x = da    
    pf = np.polynomial.polynomial.polyfit(x.time, x, 2)
    plt.plot(x.time/365, pf[2]*x.time**2 + pf[1]*x.time + pf[0] - i/2)
#     plt.axvline(40, c='grey', lw=.5)
    plt.ylabel('global mean SST [$^\circ$C]')
    plt.xlabel('time [years]')